In [1]:
import pandas as pd
import numpy as np

In [2]:
data_source = "https://archive.ics.uci.edu/ml/machine-learning-databases/cmc/cmc.data"

names = ["Wife's age", "Wife's education", "Husband's education", "Number of children ever born", "Wife's religion", 
"Wife's now working?", "Husband's occupation", "Standard-of-living index", "Media exposure", "Contraceptive method used"]

dataframe = pd.read_csv(data_source, names=names)

In [ ]:
#dataframe[:5]

In [ ]:
#dataframe.dtypes

In [3]:
for column in range(0,10):
    if column not in [0,3]:
    #if column not in [0,3,4,5,8]:
        dataframe[names[column]] = dataframe[names[column]].astype("category")

#dataframe.dtypes
# y_data = df["Contraceptive method used"].values.reshape(-1,1)

# y_data[:5]

In [4]:
y_data = dataframe.pop("Contraceptive method used")

In [5]:
dataframe.pop("Wife's age")
x_data = dataframe
x_data.insert(0, "bias", 1)
x_data = pd.get_dummies(x_data).as_matrix()

y_data = pd.get_dummies(y_data).as_matrix()
x_data

array([[  1.,   3.,   0., ...,   0.,   1.,   0.],
       [  1.,  10.,   1., ...,   1.,   1.,   0.],
       [  1.,   7.,   0., ...,   1.,   1.,   0.],
       ..., 
       [  1.,   8.,   0., ...,   1.,   1.,   0.],
       [  1.,   4.,   0., ...,   0.,   1.,   0.],
       [  1.,   1.,   0., ...,   1.,   1.,   0.]])

In [6]:
from sklearn import preprocessing as p

min_max_scaler=p.MinMaxScaler()
x_data=min_max_scaler.fit_transform(x_data)
#x_data = np.insert(x_data, 0 , 1, axis=1)

In [7]:
idx=np.random.randint(y_data.shape[0],size=int(y_data.shape[0]))
training_idx = idx[:int(y_data.shape[0]*0.8)]
test_idx = idx[int(y_data.shape[0]*0.8):]

x_training, x_test = x_data[training_idx], x_data[test_idx]
y_training, y_test = y_data[training_idx], y_data[test_idx]

In [ ]:
y_training.shape

In [8]:
import tensorflow as tf

# x = x_training.T

W = tf.Variable(tf.zeros([len(x_data[0]), len(y_data[0])]))

# W = tf.Variable(tf.random_uniform([len(y_training[0]),len(x)], -1.0, 1.0))

# X = tf.placeholder(tf.float32)
# Y = tf.placeholder(tf.float32)
                
X = tf.placeholder("float", [None, len(x_data[0])])
Y = tf.placeholder("float", [None, len(y_data[0])])


hypothesis = tf.nn.softmax(tf.matmul(X, W))

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), reduction_indices=1))

W_val = []
cost_val = []

a = tf.Variable(0.001)
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

#y_training = y_training.T

for step in range(5000000):
    sess.run(train, feed_dict={X:x_training, Y:y_training})
    if step % 500000 == 0:
        print (step, sess.run(cost, feed_dict={X:x_training, Y:y_training}), sess.run(W))

0 1.09854 [[  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  7.33941033e-06  -1.49971829e-05   7.65774712e-06]
 [  4.15959294e-05  -2.97113838e-05  -1.18845519e-05]
 [  4.15959294e-05  -5.09338097e-05   9.33785304e-06]
 [  1.92416464e-05  -4.10300127e-05   2.17883371e-05]
 [ -2.74476679e-05   1.58460571e-05   1.16015744e-05]
 [  9.62082686e-06  -3.96151745e-06  -5.65931032e-06]
 [  3.31069605e-05  -2.80135901e-05  -5.09338088e-06]
 [  2.71646859e-05  -5.00849055e-05   2.29201960e-05]
 [  5.09336223e-06  -2.37691220e-05   1.86757097e-05]
 [ -6.50820903e-06   1.13185854e-06   5.37634105e-06]
 [  8.14940649e-05  -1.06961001e-04   2.54668757e-05]
 [  2.46179934e-05  -3.31069750e-05   8.48896070e-06]
 [  5.03678493e-05  -7.27221704e-05   2.23542575e-05]
 [  4.24447762e-06   5.94227049e-06  -1.01867681e-05]
 [  2.54668867e-05  -3.98981501e-05   1.44312353e-05]
 [  4.18788877e-05  -6.67798813e-05   2.49009518e-05]
 [  3.39558551e-06  -5.09337951e-06   1.69779264e-06]
 [  2.80135846e-05

In [9]:
hypothesis_value=sess.run(hypothesis,feed_dict={X:x_test})
result=[np.argmax(predict) == np.argmax(original_value) for predict, original_value in zip(hypothesis_value,y_test)]

sum(result)/len(y_test)

0.53898305084745768